In [ ]:
import matplotlib.pylab as plt
import pandas as pd
from scipy.ndimage import gaussian_filter

# to convert from samples to s
f = 200

# select trace extraction method here
df = pd.read_parquet('./trial1_00002_data_dr12.parquet')

# individual ROIs

In [ ]:
# select of zscore ROIs or not
zroi = True

# scaling factor for y axis limits and behavior: bigger values = larger
s_ylim = 1.4
s_beh = 0.25

# smoothing (sigma for Gaussian in s)
smth_roi = .5
smth_ball = .5

# select data to plot
c_roi = [
    'roi_1', 
    'roi_2', 
    'roi_3', 
    'roi_4', 
    'roi_5', 
    'roi_6'
    ]
beh = [
    'headGrooming',
    'frontLegRub',
    'abdomenGrooming',
    'hindLegRub',
    'PER',
    'pushing',
    # 'ObL1+R1',
    # 'midLeft+hindLegRub',
    # 'midRight+hindLegRub'
]
c_beh = [ f'beh_{b}' for b in beh ]

c_ball = [ 
    'ball_x', 
    # 'ball_y', 
    # 'ball_z' 
    ]

# set up figure
cols = c_roi + c_ball
fig, axarr = plt.subplots(nrows=len(cols), figsize=(20, 2.75 * len(cols)))
x = df.index / f

for ax, c in zip(axarr, cols):
    
    # plot behavior timeline
    ax2 = ax.twinx()
    for i, c2 in enumerate(c_beh):
        y = df.loc[:, c2].values
        mask = y == 1
        offset = s_beh * i
        y = y -  offset
        color = f'C{i}'
        ax2.scatter(x[mask], y[mask], color=color, marker='|', label=c2.replace('beh_', ''))
        ax2.axhline(1 - offset, color=color, lw=0.5, ls=':')
    ax2.set_ylim(-2, 1.1)
    ax2.set_yticklabels([])
    if c == c_roi[0]:
        ax2.legend()

    # plot data      
    if c.startswith('ball'):
        y = df.loc[:, c]
        y = gaussian_filter(y.values, smth_ball * f)
        ax.plot(x, y)

    elif c.startswith('roi'):
        if zroi:
            y = df.loc[:, f'z_{c}']
        else:
            y = df.loc[:, c]
        y = gaussian_filter(y.values, smth_roi * f)
        ax.plot(x, y)
        
    ax.set_title(c)
    ax.set_xlabel('time [s]')
    ax.margins(x=0)
    ymin, ymax = ax.get_ylim()
    dy = ymax - ymin
    ax.set_ylim((ymin, ymax + s_ylim * dy))

    # optional: zero line
    ax.axhline(0, color='gray', linewidth=1)

fig.tight_layout()
fig.savefig('./data.svg')


# summarized ROIs

In [ ]:
# select of zscore ROIs or not
zroi = True

# color for ROI
color_roi = 'black'
color_vel = 'black'

# scaling factor for y axis limits and behavior: bigger values = larger
s_ylim = .25 # how much distance between ROI traces and behavior
s_beh = 0.1 # how compressed the behavoir is

# factor determining the distance in y between ROI traces
s_droi = 3

# smoothing (sigma for Gaussian in s)
smth_roi = .5
smth_ball = .5

# figure settings:
figsize = (10, 7) # x and y dimensions (lower values make text bigger)
height_ratios = (6, 2) # ratio of first to second panel

# select data to plot
c_roi = [
    'roi_1', 
    'roi_2', 
    'roi_3', 
    'roi_4', 
    'roi_5', 
    'roi_6'
    ]
beh = [
    'headGrooming',
    'frontLegRub',
    'abdomenGrooming',
    'hindLegRub',
    'PER',
    'pushing',
    # 'ObL1+R1',
    # 'midLeft+hindLegRub',
    # 'midRight+hindLegRub'
]
c_beh = [ f'beh_{b}' for b in beh ]
c_ball = 'ball_x'

# set up figure
fig, axarr = plt.subplots(nrows=2, figsize=figsize, gridspec_kw={'height_ratios':height_ratios})
x = df.index / f

ax = axarr[0]

# plot behavior timeline
ax2 = ax.twinx()
for i, c2 in enumerate(c_beh):
    y = df.loc[:, c2].values
    mask = y == 1
    offset = s_beh * i
    y = y -  offset
    color = f'C{i}'
    ax2.scatter(x[mask], y[mask], color=color, marker='|', label=c2.replace('beh_', ''))
    ax2.axhline(1 - offset, color=color, lw=0.5, ls=':')
ax2.set_ylim(-2, 1.1)
ax2.set_yticklabels([])
ax2.legend(loc='upper right')

# plot ROI traces
for i, c in enumerate(c_roi):
    if zroi:
        y = df.loc[:, f'z_{c}']
    else:
        y = df.loc[:, c]
    y = gaussian_filter(y.values, smth_roi * f)
    offset = - i * s_droi
    y = y + offset
    ax.plot(x, y, color=color_roi)

ymin, ymax = ax.get_ylim()
dy = ymax - ymin
ax.set_ylim((ymin, ymax + s_ylim * dy))

ax = axarr[1]
# plot ball velocity
y = df.loc[:, c_ball]
y = gaussian_filter(y.values, smth_ball * f)
ax.plot(x, y, color=color_vel)

for ax in axarr:
    ax.set_xlabel('time [s]')
    ax.margins(x=0)

# save
fig.tight_layout()
fig.savefig('./roi_combined.svg')